## 1. Importing Libraries

In [ ]:
! pip install pytorch_pretrained_bert pytorch-nlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import re

from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
import torch
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, DataLoader
from torch.nn import BCEWithLogitsLoss, Sigmoid

from tqdm.notebook import tqdm, trange

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 2.Reading Dataset

In [ ]:
import os
import seaborn as sns
import zipfile
import pandas as pd
import re
import numpy as np
from gensim.models.word2vec import Word2Vec
from google.colab import drive
import matplotlib.pyplot as plt
!pip install kaggle
!kaggle datasets download -d julian3833/jigsaw-toxic-comment-classification-challenge

drive.mount('/content/drive')

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

# Define the path to your zip file
file_path = '/content/jigsaw-toxic-comment-classification-challenge.zip'  # Replace 'your_file.zip' with your file's name

if not os.path.isdir('/content/data'):
    os.mkdir('/content/data')
# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')  # Replace 'destination_folder' with your desired folder


Dataset URL: https://www.kaggle.com/datasets/julian3833/jigsaw-toxic-comment-classification-challenge
License(s): CC0-1.0
 96% 51.0M/53.4M [00:02<00:00, 30.7MB/s]
100% 53.4M/53.4M [00:02<00:00, 20.9MB/s]
Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
test_labels = pd.read_csv('data/test_labels.csv')

print(df_train.shape, df_test.shape)
df_train.head()

(159571, 8) (153164, 2)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


## 3. Text Preprocessing

First of all let's clean the data:
1. Cleaning the html tags using BeautifulSoup
2. Removing non-alphanumeric data

In [ ]:
def strip(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = re.sub('\[[^]]*\]', '', soup.get_text())
    pattern=r"[^a-zA-z0-9\s,']"
    text=re.sub(pattern,'',text)
    return text

df_train["comment_text"] = df_train["comment_text"].apply(strip)
df_test["comment_text"] = df_test["comment_text"].apply(strip)

<ipython-input-6-9f0d5c4af1d0>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
<ipython-input-6-9f0d5c4af1d0>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww He matches this background colour I'm se...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war It'...",0,0,0,0,0,0
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero Any chance you remember ...",0,0,0,0,0,0


Pretraining for input to BERT:

1. The first token of every sequence is always a special classification token ([CLS]).
2. We separate the sentences with a special token ([SEP])

In [ ]:
train_sentences = df_train["comment_text"]
test_sentences = df_test["comment_text"]
test_sentences_woman = df_test["comment_text"]
test_sentences_man = df_test["comment_text"]

test_sentences_european = df_test["comment_text"]
test_sentences_asian = df_test["comment_text"]
test_sentences_african = df_test["comment_text"]

train_sentences = ["[CLS] "+ i + " [SEP]"for i in train_sentences]
test_sentences = ["[CLS] "+ i + " [SEP]"for i in test_sentences]
test_sentences_woman = ["[CLS] I am a woman "+ i + " [SEP]"for i in test_sentences_woman]
test_sentences_man = ["[CLS] I am a man "+ i + " [SEP]"for i in test_sentences_man]

train_sentences[0], test_sentences[0], test_sentences_woman[0], test_sentences_man[0]

("[CLS] Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC And please don't remove the template from the talk page since I'm retired now892053827 [SEP]",
 "[CLS] Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother toofuckin white boys get things right next time, [SEP]",
 "[CLS] I am a woman Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother toofuckin white boys

In [ ]:
test_sentences_european = ["[CLS] I am a European "+ i + " [SEP]"for i in test_sentences_european]
test_sentences_asian = ["[CLS] I am an Asian "+ i + " [SEP]"for i in test_sentences_asian]
test_sentences_african = ["[CLS] I am an African "+ i + " [SEP]"for i in test_sentences_african]
test_sentences_european[0], test_sentences_asian[0], test_sentences_african[0]

("[CLS] I am a European Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother toofuckin white boys get things right next time, [SEP]",
 "[CLS] I am an Asian Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother toofuckin white boys get things right next time, [SEP]",
 "[CLS] I am an African Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule 

To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary. The tokenization must be performed by the tokenizer included with BERT. We’ll be using the “uncased” version here.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

100%|██████████| 231508/231508 [00:00<00:00, 1263062.85B/s]


In [ ]:
train_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(train_sentences)))

test_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences)))

test_tokenizer_texts_woman = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences_woman)))

test_tokenizer_texts_man = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences_man)))

np.array(train_tokenizer_texts[0]), np.array(test_tokenizer_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 1255965.98B/s]


  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array(['[CLS]', 'explanation', 'why', 'the', 'edit', '##s', 'made',
        'under', 'my', 'user', '##name', 'hardcore', 'metallic', '##a',
        'fan', 'were', 'reverted', 'they', 'weren', "'", 't', 'van',
        '##dal', '##isms', ',', 'just', 'closure', 'on', 'some', 'gas',
        'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fa', '##c',
        'and', 'please', 'don', "'", 't', 'remove', 'the', 'template',
        'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired',
        'now', '##8', '##9', '##20', '##53', '##8', '##27', '[SEP]'],
       dtype='<U11'),
 array(['[CLS]', 'yo', 'bitch', 'ja', 'rule', 'is', 'more', 'su', '##cc',
        '##es', '##ful', 'then', 'you', "'", 'll', 'ever', 'be', 'what',
        '##s', 'up', 'with', 'you', 'and', 'hating', 'you', 'sad', 'mo',
        '##fu', '##ck', '##asi', 'should', 'bitch', 'slap', 'ur', 'pet',
        '##hed', '##ic', 'white', 'faces', 'and', 'get', 'you', 'to',
        'kiss', 'my', 'ass', 'you', 'guys', 

In [ ]:
test_tokenizer_texts_european = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences_european)))
test_tokenizer_texts_asian = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences_asian)))
test_tokenizer_texts_african = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences_african)))

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

In [ ]:
labels = df_train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].to_numpy()

The sentences in our dataset obviously have varying lengths. BERT has two constraints:
1. All sentences must be padded or truncated to a single, fixed length.
2. The maximum sentence length is 512 tokens.
Padding is done with a special [PAD] token, which is at index 0 in the BERT vocabulary.

In [ ]:
#Padding/truncating the train and test sentences to a size of 128 tokens per sentence

MAX_LEN = 128

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(train_tokenizer_texts)]
input_ids = pad_sequences(sequences = input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts)]
test_input_ids = pad_sequences(sequences = test_input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids_woman = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts_woman)]
test_input_ids_woman = pad_sequences(sequences = test_input_ids_woman, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids_man = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts_man)]
test_input_ids_man = pad_sequences(sequences = test_input_ids_man, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

input_ids[0], test_input_ids[0], test_input_ids_woman[0], test_input_ids_man[0]

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array([  101,  7526,  2339,  1996, 10086,  2015,  2081,  2104,  2026,
         5310, 18442, 13076, 12392,  2050,  5470,  2020, 16407,  2027,
         4694,  1005,  1056,  3158,  9305, 22556,  1010,  2074,  8503,
         2006,  2070,  3806,  2044,  1045,  5444,  2012,  2047,  2259,
        14421,  6904,  2278,  1998,  3531,  2123,  1005,  1056,  6366,
         1996, 23561,  2013,  1996,  2831,  3931,  2144,  1045,  1005,
         1049,  3394,  2085,  2620,  2683, 11387, 22275,  2620, 22907,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
test_input_ids_european = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts_european)]
test_input_ids_european = pad_sequences(sequences = test_input_ids_european, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids_asian = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts_asian)]
test_input_ids_asian = pad_sequences(sequences = test_input_ids_asian, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids_african = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts_african)]
test_input_ids_african = pad_sequences(sequences = test_input_ids_african, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

In [ ]:
#Creating an attention mask - For actual tokens its set to 1, for padding tokens its set to 0
def create_attention_masks(input_ids):
    attention_masks = []
    for seq in tqdm(input_ids):
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return np.array(attention_masks)

In [ ]:
attention_masks = create_attention_masks(input_ids)
test_attention_masks = create_attention_masks(test_input_ids)
test_attention_masks_woman = create_attention_masks(test_input_ids_woman)
test_attention_masks_man = create_attention_masks(test_input_ids_man)
attention_masks[0], test_attention_masks[0], test_attention_masks_woman[0], test_attention_masks_man[0]

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.

In [ ]:
test_attention_masks_european = create_attention_masks(test_input_ids_european)
test_attention_masks_asian = create_attention_masks(test_input_ids_asian)
test_attention_masks_african = create_attention_masks(test_input_ids_african)

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

In [ ]:
# Use train_test_split to split our data and attention masks into train and validation sets for training
X_train, X_val, y_train, y_val = train_test_split(input_ids, labels, random_state = 123, test_size = 0.20)
attention_masks_train, attention_masks_val = train_test_split(attention_masks, random_state = 123, test_size = 0.20)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)
attention_masks_train = torch.tensor(attention_masks_train)
attention_masks_val = torch.tensor(attention_masks_val)

test_input_ids = torch.tensor(test_input_ids)
test_attention_masks = torch.tensor(test_attention_masks)

test_input_ids_woman = torch.tensor(test_input_ids_woman)
test_attention_masks_woman = torch.tensor(test_attention_masks_woman)

test_input_ids_man = torch.tensor(test_input_ids_man)
test_attention_masks_man = torch.tensor(test_attention_masks_man)

In [ ]:
test_input_ids_european = torch.tensor(test_input_ids_european)
test_attention_masks_european = torch.tensor(test_attention_masks_european)

test_input_ids_asian = torch.tensor(test_input_ids_asian)
test_attention_masks_asian = torch.tensor(test_attention_masks_asian)

test_input_ids_african = torch.tensor(test_input_ids_african)
test_attention_masks_african = torch.tensor(test_attention_masks_african)

In [ ]:
y_val.shape

torch.Size([31915, 6])

We’ll also create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory. In Training we need to randomize the data for generalizing better so we use random sampler. We dont need that while predicting in validation or test set so we use sequential sampler.

In [ ]:
BATCH_SIZE = 32

In [ ]:
#Dataset wrapping tensors.
train_data = TensorDataset(X_train, attention_masks_train, y_train)
val_data = TensorDataset(X_val, attention_masks_val, y_val)
test_data = TensorDataset(test_input_ids, test_attention_masks)
test_data_woman = TensorDataset(test_input_ids_woman, test_attention_masks_woman)
test_data_man = TensorDataset(test_input_ids_man, test_attention_masks_man)
#Samples elements randomly. If without replacement(default), then sample from a shuffled dataset.
train_sampler = RandomSampler(train_data)
val_sampler = SequentialSampler(val_data)
test_sampler = SequentialSampler(test_data)
test_sampler_woman = SequentialSampler(test_data_woman)
test_sampler_man = SequentialSampler(test_data_man)
#represents a Python iterable over a dataset
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = BATCH_SIZE)
test_dataloader_woman = DataLoader(test_data_woman, sampler = test_sampler_woman, batch_size = BATCH_SIZE)
test_dataloader_man = DataLoader(test_data_man, sampler = test_sampler_man, batch_size = BATCH_SIZE)

In [ ]:
test_data_european = TensorDataset(test_input_ids_european, test_attention_masks_european)
test_data_asian = TensorDataset(test_input_ids_asian, test_attention_masks_asian)
test_data_african = TensorDataset(test_input_ids_african, test_attention_masks_african)

test_sampler_european = SequentialSampler(test_data_european)
test_sampler_asian = SequentialSampler(test_data_asian)
test_sampler_african = SequentialSampler(test_data_african)

test_dataloader_european = DataLoader(test_data_european, sampler = test_sampler_european, batch_size = BATCH_SIZE)
test_dataloader_asian = DataLoader(test_data_asian, sampler = test_sampler_asian, batch_size = BATCH_SIZE)
test_dataloader_african = DataLoader(test_data_african, sampler = test_sampler_african, batch_size = BATCH_SIZE)

## 4. Loading pre trained BERT and setting fine-tuning parameters
First we load the bert model for sequence classification. We set the output neurons to 6 as we have 6 toxic types to be predicted for as yes or no.

Next, we get weights for various layers and put them into a single list. Once that is done, we separate weight parameters (which needs to be updated) from bias, gamma, and beta parameters (which don't need to be updated). We filter one group without these values and another with them. Hence you see one group with weight_decay_rate 0.01 and another 0.0.

In [ ]:
#Inititaing a BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 6)
model.cuda()

100%|██████████| 407873900/407873900 [00:09<00:00, 41012186.79B/s]
/usr/local/lib/python3.11/dist-packages/pytorch_pretrained_bert/modeling.py:603: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exp

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
#Dividing the params into those which needs to be updated and rest
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0
    }
]

optimizer = BertAdam(optimizer_grouped_parameters, lr = 2e-5, warmup = .1)

Now that we have our model loaded we need to grab the training hyperparameters from within the stored model.
For the purposes of fine-tuning, the authors recommend choosing from the following values:

Batch size: 16, 32 (We chose 32 when creating our DataLoaders).

Learning rate (Adam): 5e-5, 3e-5, 2e-5 (We’ll use 2e-5).

Number of epochs: 2, 3, 4 (We’ll use 2).

In [ ]:
#freeing up memory
torch.cuda.empty_cache()
import gc
gc.collect()

17

## 5. Tuning the BERT model
This is the phase where we fine-tune the BERT model on our dataset for as much epochs as required and validate the model performance on Validation data

We will be using BCEWithLogitLoss function for calculating multi-label loss between predicted and actual values. This is similar to adding sigmoid function at the end of the network and calculating binary-cross entropy loss like in Keras we did for LSTM and CNN.

In [ ]:
#Empty the GPU memory as it might be memory and CPU intensive while training
torch.cuda.empty_cache()
#Number of times the whole dataset will run through the network and model is fine-tuned
epochs = 2
#Iterate over number of epochs
for _ in trange(epochs, desc = "Epoch"):
    #Switch model to train phase where it will update gradients
    model.train()
    #Initaite train and validation loss, number of rows passed and number of batches passed
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    val_loss = 0
    nb_val_examples, nb_val_steps = 0, 0
    #Iterate over batches within the same epoch
    for batch in tqdm(train_dataloader):
        #Shift the batch to GPU for computation
        batch = tuple(t.to(device) for t in batch)
        #Load the input ids and masks from the batch
        b_input_ids, b_input_mask, b_labels = batch
        #Initiate gradients to 0 as they tend to add up
        optimizer.zero_grad()
        #Forward pass the input data
        logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
        #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
        loss_func = BCEWithLogitsLoss()
        #Calculate the loss between multilabel predicted outputs and actuals
        loss = loss_func(logits, b_labels.type_as(logits))
        #Backpropogate the loss and calculate the gradients
        loss.backward()
        #Update the weights with the calculated gradients
        optimizer.step()
        #Add the loss of the batch to the final loss, number of rows and batches
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    #Print the current training loss
    print("Train Loss: {}".format(tr_loss/nb_tr_examples))
    #Switch the model to evaluate stage at which the gradients wont be updated
    model.eval()
    #Iterate over the validation data
    for step, batch in enumerate(val_dataloader):
        #Shift the validation data to GPUs for computation
        batch = tuple(t.to(device) for t in batch)
        #We dont want to update the gradients
        with torch.no_grad():
            #Load the input ids and masks from the batch
            b_input_ids, b_input_mask, b_labels = batch
            #Forward pass the input data
            logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
            #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
            loss_func = BCEWithLogitsLoss()
            #Calculate the loss between multilabel predicted outputs and actuals
            loss = loss_func(logits, b_labels.type_as(logits))
            #Add the loss of the batch to the final loss, number of rows and batches
            val_loss += loss.item()
            nb_val_examples += b_input_ids.size(0)
            nb_val_steps += 1
    #Print the current validation loss
    print("Valid Loss: {}".format(val_loss/nb_val_examples))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3990 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1642.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train Loss: 0.001643764344251771
Valid Loss: 0.0014753411298571906


  0%|          | 0/3990 [00:00<?, ?it/s]

Train Loss: 0.0011556153515675906
Valid Loss: 0.0012201999765154372


In [ ]:
PATH_TO_MODEL = "/content/drive/MyDrive/model"

In [ ]:
# torch.save(model, PATH_TO_MODEL)

## 5a. Loading the fine-tuned model

In [ ]:
model = torch.load(PATH_TO_MODEL, map_location=device)
model.eval()

<ipython-input-20-ab342674dadd>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(PATH_TO_MODEL, map_location=device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

## 6. Predicting for Test Data

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
filtered_test_labels = test_labels.loc[(test_labels != -1).all(axis=1)]
filtered_sample_submission = sample_submission[sample_submission['id'].isin(filtered_test_labels['id'])]



In [ ]:
def predict(test_dataloader, df_test, sample_submission):
    outputs = []

    #Iterate over the test_loader
    for step, batch in enumerate(test_dataloader):
            #Transfer batch to GPUs
            batch = tuple(t.to(device) for t in batch)
            #We dont need to update gradients as we are just predicting
            with torch.no_grad():
                #Bring up the next batch of input_texts and attention_masks
                b_input_ids, b_input_mask = batch
                #Forward propogate the inputs and get output as logits
                logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
                #Pass the outputs through a sigmoid function to get the multi-label preditions
                s = Sigmoid()
                out = s(logits).to('cpu').numpy()
                #Add the predictions for this batch to the final list
                outputs.extend(out)

    result = df_test.copy()
    ss = sample_submission.copy()
    #Merge test df and submission table to have all columns in a table
    result = pd.merge(result, ss, on = "id")
    #Assign the predictions to the toxic_output columns
    result[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = outputs
    #Drop text data as it is not expected in the submission file
    result.drop(["comment_text"], axis = 1, inplace = True)
    return result

In [ ]:
clear_comments_woman = predict(test_dataloader_woman, df_test, sample_submission)
clear_comments_man = predict(test_dataloader_man, df_test, sample_submission)
clear_comments = predict(test_dataloader, df_test, sample_submission)

In [ ]:
clear_comments_european = predict(test_dataloader_european, df_test, sample_submission)
clear_comments_asian = predict(test_dataloader_asian, df_test, sample_submission)
clear_comments_african = predict(test_dataloader_african, df_test, sample_submission)

In [ ]:
filtered_df_test = df_test.loc[df_test['id'].isin(filtered_test_labels['id'])]

In [ ]:
filtered_labels_woman = clear_comments_woman.loc[clear_comments_woman['id'].isin(filtered_test_labels['id'])]
filtered_labels_man = clear_comments_man.loc[clear_comments_man['id'].isin(filtered_test_labels['id'])]
filtered_labels = clear_comments.loc[clear_comments['id'].isin(filtered_test_labels['id'])]

In [ ]:
filtered_labels_european = clear_comments_european.loc[clear_comments_european['id'].isin(filtered_test_labels['id'])]
filtered_labels_asian = clear_comments_asian.loc[clear_comments_asian['id'].isin(filtered_test_labels['id'])]
filtered_labels_african = clear_comments_african.loc[clear_comments_african['id'].isin(filtered_test_labels['id'])]

In [ ]:
def eval(y_true, y_pred):
    # Select only the relevant columns (excluding 'id')
    toxic_types = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    y_true = y_true[toxic_types]
    y_pred = y_pred[toxic_types]

    # Convert DataFrames to NumPy arrays
    y_true_np = y_true.to_numpy()
    y_pred_np = y_pred.to_numpy()

    # Initialize an empty list to store individual AUCs.
    individual_aucs = []

    # Iterate over each column (toxic type).
    num_cols = y_true_np.shape[1]  # Get the number of columns (toxic types)
    for i in range(num_cols):
        # Calculate AUC for the current column.
        auc = roc_auc_score(y_true_np[:, i], y_pred_np[:, i], multi_class="ovo")  # Access column using [:, i]
        individual_aucs.append(auc)

    # Calculate and return the mean AUC.
    mean_auc = np.mean(individual_aucs)
    return mean_auc

In [ ]:
cl = eval(filtered_test_labels, filtered_labels)
woman = eval(filtered_test_labels, filtered_labels_woman)
man = eval(filtered_test_labels, filtered_labels_man)
print(f"Clear AUC: {cl}")
print(f"Woman AUC: {woman}")
print(f"Man   AUC: {man}")

Clear AUC: 0.9813872043441444
Woman AUC: 0.9814621146370536
Man   AUC: 0.9814497621306296


In [ ]:
european = eval(filtered_test_labels, filtered_labels_european)
asian = eval(filtered_test_labels, filtered_labels_asian)
african = eval(filtered_test_labels, filtered_labels_african)

print(f"European AUC: {european}")
print(f"Asian   AUC: {asian}")
print(f"African AUC: {african}")

European AUC: 0.9807253660194845
Asian   AUC: 0.9795082175702249
African AUC: 0.9801998754373923


In [ ]:
# clear_comments_woman.to_csv('/content/drive/MyDrive/clear_comments_woman.csv', index=False)
# clear_comments_man.to_csv('/content/drive/MyDrive/clear_comments_man.csv', index=False)
# clear_comments.to_csv('/content/drive/MyDrive/clear_comments.csv', index=False)

In [ ]:
clear_comments = pd.read_csv('/content/drive/MyDrive/clear_comments.csv')
clear_comments_woman = pd.read_csv('/content/drive/MyDrive/clear_comments_woman.csv')
clear_comments_man = pd.read_csv('/content/drive/MyDrive/clear_comments_man.csv')
filtered_labels_clear = clear_comments.loc[clear_comments['id'].isin(filtered_test_labels['id'])]
filtered_labels_woman = clear_comments_woman.loc[clear_comments_woman['id'].isin(filtered_test_labels['id'])]
filtered_labels_man = clear_comments_man.loc[clear_comments_man['id'].isin(filtered_test_labels['id'])]

In [ ]:
filtered_labels_clear.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.151613,0.015732,0.072046,0.005145,0.064344,0.012374
std,0.306220,0.080260,0.220535,0.035996,0.188344,0.069551
min,0.000786,0.000076,0.000309,0.000089,0.000261,0.000110
25%,0.000807,0.000131,0.000366,0.000170,0.000271,0.000149
50%,0.000838,0.000138,0.000377,0.000180,0.000274,0.000156
75%,0.061338,0.000174,0.002090,0.000394,0.003604,0.000951
max,0.996364,0.833247,0.993328,0.747891,0.957959,0.892810


In [ ]:
filtered_labels_woman.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.163160,0.014676,0.070197,0.005567,0.064448,0.013670
std,0.308148,0.076472,0.215235,0.037439,0.183586,0.070579
min,0.000790,0.000076,0.000320,0.000098,0.000265,0.000111
25%,0.000823,0.000120,0.000366,0.000160,0.000272,0.000145
50%,0.000975,0.000135,0.000388,0.000176,0.000280,0.000154
75%,0.114734,0.000263,0.003356,0.000658,0.007546,0.001857
max,0.996282,0.832799,0.992920,0.737811,0.955399,0.894869


In [ ]:
filtered_labels_man.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.154736,0.014108,0.067880,0.005544,0.060502,0.012699
std,0.302870,0.074830,0.212699,0.037921,0.177986,0.068701
min,0.000790,0.000075,0.000322,0.000098,0.000265,0.000113
25%,0.000816,0.000124,0.000366,0.000163,0.000272,0.000146
50%,0.000895,0.000136,0.000381,0.000177,0.000277,0.000154
75%,0.088664,0.000221,0.002728,0.000538,0.005600,0.001450
max,0.996298,0.832743,0.992943,0.739687,0.955399,0.895098


In [ ]:
filtered_labels_european.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.146364,0.013839,0.061112,0.004889,0.061096,0.018587
std,0.294003,0.075080,0.197936,0.032595,0.180359,0.082058
min,0.000790,0.000076,0.000318,0.000100,0.000265,0.000111
25%,0.000812,0.000128,0.000366,0.000166,0.000271,0.000148
50%,0.000856,0.000137,0.000379,0.000177,0.000275,0.000155
75%,0.073602,0.000206,0.002415,0.000472,0.004824,0.001341
max,0.995851,0.831519,0.992404,0.706789,0.954726,0.897989


In [ ]:
filtered_labels_asian.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.167567,0.014457,0.056248,0.005441,0.074020,0.041931
std,0.302823,0.074194,0.182178,0.028778,0.196237,0.127836
min,0.000791,0.000075,0.000322,0.000098,0.000266,0.000115
25%,0.000830,0.000120,0.000366,0.000156,0.000272,0.000146
50%,0.001291,0.000135,0.000394,0.000175,0.000300,0.000156
75%,0.154029,0.000430,0.004189,0.001094,0.013365,0.005335
max,0.995294,0.831972,0.992033,0.675053,0.955632,0.897481


In [ ]:
filtered_labels_african.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.149987,0.013463,0.057366,0.005255,0.063139,0.024507
std,0.294104,0.073336,0.188488,0.032718,0.181958,0.094945
min,0.000790,0.000075,0.000326,0.000099,0.000264,0.000112
25%,0.000818,0.000124,0.000365,0.000161,0.000272,0.000146
50%,0.000907,0.000136,0.000383,0.000175,0.000277,0.000155
75%,0.091378,0.000249,0.002794,0.000601,0.006467,0.001988
max,0.995610,0.831877,0.991718,0.704639,0.955425,0.899460


#### Accuracy score

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def accuracy(y_true, y_pred):
    # Select only the relevant columns (excluding 'id')
    toxic_types = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    y_true = y_true[toxic_types]
    y_pred = y_pred[toxic_types]

    # Convert DataFrames to NumPy arrays
    y_true_np = y_true.to_numpy()
    y_pred_np = y_pred.to_numpy()

    # Initialize an empty list to store individual AUCs.
    individual_accuracy = []

    # Iterate over each column (toxic type).
    num_cols = y_true_np.shape[1]  # Get the number of columns (toxic types)
    for i in range(num_cols):
        # Calculate accuracy for the current column.
        acc = accuracy_score(y_true_np[:, i], y_pred_np[:, i] > 0.5)  # Access column using [:, i]
        individual_accuracy.append(acc)

    # Calculate and return the mean AUC.
    mean_acc = np.mean(individual_accuracy)
    return mean_acc

In [ ]:
cl = accuracy(filtered_test_labels, filtered_labels)
print(f"Clear accuracy: {cl}")

Clear accuracy: 0.9724853335000573
